In [8]:
library("eurostat")
library("flagr")
library("tidyr")
flags <- spread(test_data[, c(1:3)], key = time, value = flags)

**hierarchy method**

In [6]:
propagate_flag(flags[, c(2:ncol(flags))],"hierarchy","estat")
propagate_flag(flags[, c(2:ncol(flags))],"hierarchy","puebscd")
propagate_flag(flags[, c(2:ncol(flags))],"hierarchy",c("u","c","d","e","p","s","b"))

$`2009-01-01`
[1] "e"

$`2010-01-01`
[1] "b"

$`2011-01-01`
[1] "e"

$`2012-01-01`
[1] "b"

$`2013-01-01`
[1] "b"

$`2014-01-01`
[1] "b"

$`2015-01-01`
[1] "b"

$`2009-01-01`
[1] "p"

$`2010-01-01`
[1] "p"

$`2011-01-01`
[1] "p"

$`2012-01-01`
[1] "p"

$`2013-01-01`
[1] "p"

$`2014-01-01`
[1] "p"

$`2015-01-01`
[1] "p"

$`2009-01-01`
[1] "e"

$`2010-01-01`
[1] "p"

$`2011-01-01`
[1] "c"

$`2012-01-01`
[1] "u"

$`2013-01-01`
[1] "u"

$`2014-01-01`
[1] "u"

$`2015-01-01`
[1] "u"

**frequency method**

In [7]:
propagate_flag(flags[, c(2:ncol(flags))],"frequency")

$`2009-01-01`
[1] "p"

$`2010-01-01`
[1] "p"

$`2011-01-01`
[1] "e"

$`2012-01-01`
[1] "e" "u"

$`2013-01-01`
[1] "u"

$`2014-01-01`
[1] "p"

$`2015-01-01`
[1] "p"

**weighted method**

In [9]:
flags<-flags[, c(2:ncol(flags))]
weights <- spread(test_data[, c(1, 3:4)], key = time, value = values)
weights<-weights[, c(2:ncol(weights))]

propagate_flag(flags,"weighted",flag_weights=weights,threshold=0.1)

$`2009-01-01`
$`2009-01-01`[[1]]
[1] NA

$`2009-01-01`[[2]]
[1] NA


$`2010-01-01`
$`2010-01-01`[[1]]
[1] NA

$`2010-01-01`[[2]]
[1] NA


$`2011-01-01`
$`2011-01-01`[[1]]
[1] "e"

$`2011-01-01`[[2]]
[1] "0.133211654011488"


$`2012-01-01`
$`2012-01-01`[[1]]
[1] "u"

$`2012-01-01`[[2]]
[1] "0.124148695828608"


$`2013-01-01`
$`2013-01-01`[[1]]
[1] NA

$`2013-01-01`[[2]]
[1] NA


$`2014-01-01`
$`2014-01-01`[[1]]
[1] "p"

$`2014-01-01`[[2]]
[1] "0.174097815337322"


$`2015-01-01`
$`2015-01-01`[[1]]
[1] "p"

$`2015-01-01`[[2]]
[1] "0.335586943761588"

**Use 'flagr' with the tables from Eurostat database**

In [31]:
tbl_name="ilc_peps01"
data<-get_eurostat(tbl_name,keepFlags=T)
data<-data[data$geo %in% eu_countries$code,]
cn<-ncol(data)
data$id<-do.call(paste0, data[c(1:(cn-4))])
bdown<-unique(data$id)
cond<-bdown[1]
cnames<-colnames(data[,c(1:(cn-4))])
ss<-data[data$id==cond,c((cn-3):(cn))]

flags <- spread(ss[, c(1:3)], key = time, value = flags)
hierarchy_flag<-propagate_flag(flags[, c(2:ncol(flags))],"hierarchy","estat")
frequency_flag<-propagate_flag(flags[, c(2:ncol(flags))],"frequency")
flags<-flags[, c(2:ncol(flags))]
weights <- spread(ss[, c(1, 3:4)], key = time, value = values)
weights<-weights[, c(2:ncol(weights))]
weighted_flag<-propagate_flag(flags,"weighted",flag_weights=weights,threshold=0.1)


out<-cbind(unique(data[data$id==cond,c(1:(cn-4))]),data.frame(unlist(hierarchy_flag)),data.frame(unlist(frequency_flag)),t(sapply(weighted_flag,unlist)))
colnames(out)<-c(cnames,"hierarchy","frequency","weighted","max_weight")
out
add_flag<-function(cond){
}

#sapply(bdown,FUN=add_flag)


Reading cache file /tmp/RtmpMfY7Sy/eurostat/ilc_peps01_date_code_TT.rds
Table  ilc_peps01  read from cache file:  /tmp/RtmpMfY7Sy/eurostat/ilc_peps01_date_code_TT.rds


,unit,age,sex,hierarchy,frequency,weighted,max_weight
2003-01-01,PC,TOTAL,F,NA,NA,NA,NA
2004-01-01,PC,TOTAL,F,NA,NA,NA,NA
2005-01-01,PC,TOTAL,F,NA,NA,NA,NA
2006-01-01,PC,TOTAL,F,NA,NA,NA,NA
2007-01-01,PC,TOTAL,F,NA,NA,NA,NA
2008-01-01,PC,TOTAL,F,b,b,2,0.330540858318636
2009-01-01,PC,TOTAL,F,NA,NA,NA,NA
2010-01-01,PC,TOTAL,F,NA,NA,NA,NA
2011-01-01,PC,TOTAL,F,b,b,NA,NA
2012-01-01,PC,TOTAL,F,b,b,NA,NA
